In [8]:
from util import *
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

source_initial_data_file = "C:/Users/parth/Desktop/workspace/data_pipeline/resources/sample_data/customers.csv"
source_updated_data_file = "C:/Users/parth/Desktop/workspace/data_pipeline/resources/sample_data/customers_full.csv"

full_overwrite_path = "C:/Users/parth/Desktop/workspace/data_pipeline/output/customers_full_overwrite"
incremental_upsert_path = "C:/Users/parth/Desktop/workspace/data_pipeline/output/customers_incremental_upsert"
incremental_insert_path = "C:/Users/parth/Desktop/workspace/data_pipeline/output/customers_incremental_with_history"

In [9]:
initial_df = pd.read_csv(source_initial_data_file)
initial_df.head()

,id,name,email,created_date,updated_date
0,1,Jammie McCamish,jmccamish0@devhub.com,2023-01-01 03:17:00,2023-01-01 03:17:00
1,2,Brocky Spurret,bspurret1@npr.org,2023-01-01 05:35:00,2023-01-01 05:35:00


In [10]:
updated_df = pd.read_csv(source_updated_data_file)
updated_df.head()

,id,name,email,created_date,updated_date
0,2,Brocky Spurret,bspurret1@npr.org,2023-01-01 05:35:00,2023-01-01 05:35:00
1,1,Jammie McCamish,jmccamish0@develophub.com,2023-01-01 03:17:00,2023-01-02 03:17:00


In [2]:
spark = get_spark_with_hudi()

In [3]:
full_overwrite_df = spark.read.format("hudi").load(full_overwrite_path)
incremental_insert_df = spark.read.format("hudi").load(incremental_insert_path)
incremental_upsert_df = spark.read.format("hudi").load(incremental_upsert_path)

In [4]:
full_overwrite_df.select("id", "name", "email", "created_date", "updated_date", "w_create_ts").toPandas().head(10)

,id,name,email,created_date,updated_date,w_create_ts
0,2,Brocky Spurret,bspurret1@npr.org,2023-01-01 05:35:00,2023-01-01 05:35:00,2023-01-02
1,1,Jammie McCamish,jmccamish0@develophub.com,2023-01-01 03:17:00,2023-01-02 03:17:00,2023-01-02


In [5]:
incremental_upsert_df.select("id", "name", "email", "created_date", "updated_date", "w_create_ts").toPandas().head(10)

,id,name,email,created_date,updated_date,w_create_ts
0,2,Brocky Spurret,bspurret1@npr.org,2023-01-01 05:35:00,2023-01-01 05:35:00,2023-01-01
1,1,Jammie McCamish,jmccamish0@develophub.com,2023-01-01 03:17:00,2023-01-02 03:17:00,2023-01-02


In [6]:
incremental_insert_df.select("id", "name", "email", "created_date", "updated_date", "w_create_ts").toPandas().head(10)

,id,name,email,created_date,updated_date,w_create_ts
0,2,Brocky Spurret,bspurret1@npr.org,2023-01-01 05:35:00,2023-01-01 05:35:00,2023-01-01
1,1,Jammie McCamish,jmccamish0@devhub.com,2023-01-01 03:17:00,2023-01-01 03:17:00,2023-01-01
2,1,Jammie McCamish,jmccamish0@develophub.com,2023-01-01 03:17:00,2023-01-02 03:17:00,2023-01-02
